In [ ]:
import pandas as pd
import numpy as np
from tqdm import tqdm
import cv2

In [ ]:
data = pd.read_csv("Ecoline-2021-09_2022-03-10.csv", sep=";")

In [ ]:
data = data.drop(columns=["FractionId"])
data = data.drop(columns=["FractionName"])
data = data.drop(columns=["SeriesName"])
data = data.drop(columns=["AllPhotoSegmentsClassified"])
data = data.drop(columns=["Question"])

In [ ]:
y = data['Points']
arr = np.zeros((len(y), 4))

for i in range(len(y)):
  arr[i][2] = 1700
  arr[i][3] = 1300

In [ ]:
def coord(y, arr):
  ss = 0
  for i in range(len(y)):
    z = y[i]
    z = z[2: len(y[i]) - 2]
    z = z.split("},{")
    for j in range(len(z)):
      tmp = z[j].split(",")
      tmp_1 = tmp[0].split(":")
      tmp_2 = tmp[1].split(":")
      x = int(tmp_1[1])
      cord_y = int(tmp_2[1])
      if x > arr[i][0]:
        arr[i][0] = x
      if x < arr[i][2]:
        arr[i][2] = x
      if cord_y > arr[i][1]:
        arr[i][1] = cord_y
      if cord_y < arr[i][3]:
        arr[i][3] = cord_y
      ss += 1
  return arr, ss

In [ ]:
aray, ss = coord(y, arr)

In [ ]:
aray

array([[ 968.,  707.,  771.,  561.],
       [1456.,  654.,  913.,  398.],
       [1100.,  606.,  820.,  451.],
       ...,
       [ 471., 1063.,  127.,  720.],
       [ 585.,  345.,  256.,   84.],
       [1203.,  247., 1014.,    0.]])

In [ ]:
pic = data['ImageFileName']
new = 'jpg'
def pictures(pic):
  for i in range(len(pic)):
    pic[i] = pic[i][:len(pic[i]) - 3] + new
    # pic[i] = pic[i] + new
  return pic

pic = pictures(pic)

C:\Users\1\AppData\Local\Temp\ipykernel_7404\1841067997.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  pic[i] = pic[i][:len(pic[i]) - 3] + new


In [ ]:
pic

0        53b812a205144b49ba0d173b4fb49b64.jpg
1        53b812a205144b49ba0d173b4fb49b64.jpg
2        53b812a205144b49ba0d173b4fb49b64.jpg
3        53b812a205144b49ba0d173b4fb49b64.jpg
4        53b812a205144b49ba0d173b4fb49b64.jpg
                         ...                 
12030    81f99790c0f54011bbdc75203aa2edf7.jpg
12031    81f99790c0f54011bbdc75203aa2edf7.jpg
12032    81f99790c0f54011bbdc75203aa2edf7.jpg
12033    81f99790c0f54011bbdc75203aa2edf7.jpg
12034    81f99790c0f54011bbdc75203aa2edf7.jpg
Name: ImageFileName, Length: 12035, dtype: object

In [ ]:
df1 = pd.DataFrame(pic)
# df1['classes'] = 0
df2 = pd.DataFrame(aray)
df = pd.concat([df1, df2], axis=1)
df = df.rename(columns={0: 'xmax', 1: 'ymax', 2: 'xmin', 3: 'ymin'})
df

,ImageFileName,xmax,ymax,xmin,ymin
0,53b812a205144b49ba0d173b4fb49b64.jpg,968.0,707.0,771.0,561.0
1,53b812a205144b49ba0d173b4fb49b64.jpg,1456.0,654.0,913.0,398.0
2,53b812a205144b49ba0d173b4fb49b64.jpg,1100.0,606.0,820.0,451.0
3,53b812a205144b49ba0d173b4fb49b64.jpg,1271.0,869.0,914.0,668.0
4,53b812a205144b49ba0d173b4fb49b64.jpg,1138.0,1052.0,809.0,768.0
...,...,...,...,...,...
12030,81f99790c0f54011bbdc75203aa2edf7.jpg,471.0,49.0,329.0,0.0
12031,81f99790c0f54011bbdc75203aa2edf7.jpg,287.0,156.0,60.0,4.0
12032,81f99790c0f54011bbdc75203aa2edf7.jpg,471.0,1063.0,127.0,720.0
12033,81f99790c0f54011bbdc75203aa2edf7.jpg,585.0,345.0,256.0,84.0


In [ ]:
df['classes'] = data['Answer']
d = {'Ничего из вышеперечисленного': 0, 'ПЭТ коричневый': 1, 'ПЭТ прозрачный': 2, 'ПЭТ зеленый': 3, 'NaN': 4, 'ПЭТ масло': 5, 'ПЭТ молочка (белая/прозрачная)': 6,
     'ПЭТ голубой': 7, 'Одноразовая посуда (пластик)': 8, 'Любая бутылка с термоусадкой': 9, 'Бутылка Бытовая химия': 10, 'Черные металлы': 11, 'Пленка (пакеты/стретч) белая/прозрачная': 12, 'Тэтрапак': 13, 'ПЭТ бирюзовый': 14, 
     'Пленка (пакеты/стретч) цветная': 15, 'ПЭТ остальные цвета': 16, 'Алюминий': 17}
df = df.replace({'classes' : d})
df = df[~np.isnan(df['classes'])].reset_index(drop=True)
df

,ImageFileName,xmax,ymax,xmin,ymin,classes
0,53b812a205144b49ba0d173b4fb49b64.jpg,968.0,707.0,771.0,561.0,0.0
1,53b812a205144b49ba0d173b4fb49b64.jpg,1456.0,654.0,913.0,398.0,1.0
2,53b812a205144b49ba0d173b4fb49b64.jpg,1100.0,606.0,820.0,451.0,2.0
3,53b812a205144b49ba0d173b4fb49b64.jpg,1271.0,869.0,914.0,668.0,3.0
4,53b812a205144b49ba0d173b4fb49b64.jpg,1138.0,1052.0,809.0,768.0,2.0
...,...,...,...,...,...,...
9613,81f99790c0f54011bbdc75203aa2edf7.jpg,785.0,925.0,449.0,640.0,10.0
9614,81f99790c0f54011bbdc75203aa2edf7.jpg,471.0,49.0,329.0,0.0,2.0
9615,81f99790c0f54011bbdc75203aa2edf7.jpg,287.0,156.0,60.0,4.0,0.0
9616,81f99790c0f54011bbdc75203aa2edf7.jpg,585.0,345.0,256.0,84.0,0.0


In [ ]:
def pars_train(da1, way):
    data = da1[:8176]
    fil = open(way, "w")
    for i in tqdm(range(len(data['ImageFileName'].unique()) - 1)):
        mask = data['ImageFileName'] == data['ImageFileName'].unique()[0]
        df_tmp, data = data[mask], data[~mask]
        data = data.rename(index = lambda x: x - df_tmp.shape[0])
        fil = open(way, "a").write(df_tmp['ImageFileName'][0] + '\n')

    fil = open(way, "a").write(data['ImageFileName'][0] + '\n')
    
def pars_val(da1, way):
    data = da1[8176:].reset_index(drop=True)
    fil = open(way, "w")
    for i in tqdm(range(len(data['ImageFileName'].unique()) - 1)):
        mask = data['ImageFileName'] == data['ImageFileName'].unique()[0]
        df_tmp, data = data[mask], data[~mask]
        data = data.rename(index = lambda x: x - df_tmp.shape[0])
        fil = open(way, "a").write(df_tmp['ImageFileName'][0] + '\n')
    
    fil = open(way, "a").write(data['ImageFileName'][0] + '\n')
    
def pars(da1, q, p):
    data = da1
    for i in tqdm(range(len(data['ImageFileName'].unique()) - 1)):
        mask = data['ImageFileName'] == data['ImageFileName'].unique()[0]
        df_tmp, data = data[mask], data[~mask]
        y = np.array(df_tmp[['xmax', 'ymax', 'xmin', 'ymin', 'classes']]).tolist()
        data = data.rename(index = lambda x: x - df_tmp.shape[0])
        tit3 = p + df_tmp['ImageFileName'][0]
        tit4 = q + df_tmp['ImageFileName'][0][:len(df_tmp['ImageFileName'][0]) - 3] + 'txt'
        image = cv2.imread(df_tmp['ImageFileName'][0])
        image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
        fil = open(tit4, "w")
        for j in range(len(y)):
            st2 = str(y[j][4]) + ' ' + str(y[j][2]) + ' ' +  str(y[j][3]) + ' ' +  str(y[j][0]) + ' ' + str(y[j][1]) + '\n'
            fil = open(tit4, "a").write(st2)
        cv2.imwrite(tit3, image)
    
    tit3 = p + data['ImageFileName'][0]
    tit4 = q + data['ImageFileName'][0][:len(data['ImageFileName'][0]) - 3] + 'txt'
    image = cv2.imread(data['ImageFileName'][0])
    image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
    fil = open(tit4, "w")
    y = np.array(data[['xmax', 'ymax', 'xmin', 'ymin', 'classes']]).tolist()
    for j in range(len(y)):
        st2 = str(y[j][4]) + ' ' + str(y[j][2]) + ' ' +  str(y[j][3]) + ' ' +  str(y[j][0]) + ' ' + str(y[j][1]) + '\n'
    fil = open(tit4, "a").write(st2)

In [ ]:
q = "d:/VKR/parser/kitti_tiny/train.txt"

# pars_train(df, q)

100%|███████████████████████████████████████████████████████████████████████████████| 920/920 [00:07<00:00, 115.46it/s]


In [ ]:
q = "d:/VKR/parser/kitti_tiny/val.txt"

# pars_val(df, q)

100%|███████████████████████████████████████████████████████████████████████████████| 153/153 [00:00<00:00, 323.19it/s]


In [ ]:
q = "d:/VKR/parser/kitti_tiny/training/label_2/"
p = "d:/VKR/parser/kitti_tiny/training/image_2/"

# pars(df, q, p)

100%|██████████████████████████████████████████████████████████████████████████████| 1073/1073 [03:50<00:00,  4.66it/s]


In [ ]:
df.head(15)

,ImageFileName,xmax,ymax,xmin,ymin,classes
0,53b812a205144b49ba0d173b4fb49b64.jpg,968.0,707.0,771.0,561.0,0.0
1,53b812a205144b49ba0d173b4fb49b64.jpg,1456.0,654.0,913.0,398.0,1.0
2,53b812a205144b49ba0d173b4fb49b64.jpg,1100.0,606.0,820.0,451.0,2.0
3,53b812a205144b49ba0d173b4fb49b64.jpg,1271.0,869.0,914.0,668.0,3.0
4,53b812a205144b49ba0d173b4fb49b64.jpg,1138.0,1052.0,809.0,768.0,2.0
5,53b812a205144b49ba0d173b4fb49b64.jpg,870.0,1114.0,358.0,821.0,2.0
6,53b812a205144b49ba0d173b4fb49b64.jpg,597.0,1214.0,393.0,1086.0,1.0
7,53b812a205144b49ba0d173b4fb49b64.jpg,785.0,1200.0,604.0,1092.0,2.0
8,53b812a205144b49ba0d173b4fb49b64.jpg,1036.0,1215.0,708.0,1018.0,2.0
9,53b812a205144b49ba0d173b4fb49b64.jpg,819.0,669.0,471.0,440.0,0.0
